# Education

## Setup

In [1]:
from matplotlib.colors import ListedColormap 
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from lib import formatting as sd_formatting
import pyodbc 
import toml
import config_logging
import logging
import urllib.request
import pandas as pd
import colorcet as cc

log = logging.getLogger(__name__)
external_ip = urllib.request.urlopen('https://ident.me').read().decode('utf8')

properties = toml.load("./.streamlit/secrets.toml")

database_props = properties[properties["database"]["flavour"]] 


2022-08-25 13:09:27,033 [INFO] numexpr.utils: NumExpr defaulting to 4 threads.


## Reload

In [20]:
### Set up
%load_ext autoreload

%autoreload 3
from data.daos import dao_facade_local as dao_fac
from lib import db_tools as db_tools
from lib import masters_data_analytics_lib as mlib
from lib import stats as stats

db_conn = None
db_conn = db_tools.get_db_conn(database_props)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-08-24 13:40:26,842 [INFO] lib.db_tools: DATABASE CONNECTIVITY


## Build the Data

### Search Critera & DAO Access to get all required data

In [21]:
year_from = 2012
year_to   = 2020
city      = "London"

borough   = "Islington"
ward_name = "Holloway"

# ## 27% Black African Carribean at Borough Level
# borough   = "Lewisham"
# ward_name = "Bellingham"



# borough   = "Westminster"    # Highest
# borough   = "Brent"          # Middle
# borough   = "City of London" # Lowest
# ward_name = "Coleman Street"

## We may not have data in the date range we have chosen
## this is true for ALL so fix it Neal

education_year_from = year_from
education_year_to   = year_to

# What do we have? We are only using the max for now
education_min_max_year_df = dao_fac.qualifications_min_max_year(db_conn)

log.debug(education_min_max_year_df)

education_year_min = education_min_max_year_df["MIN_YEAR"].values[0]
education_year_max = education_min_max_year_df["MAX_YEAR"].values[0]

log.debug(f"education_year_min:{education_year_min}")
log.debug(f"education_year_max:{education_year_max}")

education_year_from_orig = education_year_from
education_year_to_orig = education_year_to

if education_year_from < int(education_year_min):
    education_year_from = int(education_year_min)
elif education_year_from > int(education_year_max):
    education_year_from = int(education_year_max)

if education_year_to > int(education_year_max):
    education_year_to = int(education_year_max)
elif education_year_to < int(education_year_min):
    education_year_to = int(education_year_min)

log.debug(f"education orig_year_to    :{education_year_to_orig}")
log.debug(f"education search_year_to  :{education_year_to}")

search_term = {"year_from" : education_year_from,
                 "year_to" : education_year_to,
                 "borough" : borough,
               "ward_name" : ward_name}

## City
education_average_year_df         = dao_fac.education_ratio_average_years(db_conn, search_term)
## Borough
education_by_borough_year_df      = dao_fac.education_ratio_by_borough_years(db_conn, search_term)
## Ward
education_by_borough_ward_year_df = dao_fac.education_ratio_by_borough_ward_years(db_conn, search_term)


2022-08-24 13:40:26,991 [DEBUG] __main__:    MAX_YEAR  MIN_YEAR
0      2011      2011
2022-08-24 13:40:26,994 [DEBUG] __main__: education_year_min:2011
2022-08-24 13:40:26,996 [DEBUG] __main__: education_year_max:2011
2022-08-24 13:40:26,997 [DEBUG] __main__: education orig_year_to    :2020
2022-08-24 13:40:26,999 [DEBUG] __main__: education search_year_to  :2011


In [22]:
## Education Search Range Narrative
education_narrative_01 = ""
education_in_not_in = "in" if ((education_year_to >= education_year_from_orig) &
                            (education_year_to <= education_year_to)) else "outside"

education_search_range = f"of {education_year_from_orig} to {education_year_to_orig}" if education_year_from_orig != education_year_to_orig else f"{education_year_to_orig}"
education_narrative_search_criters = f"Using the latest qualifications data from {education_year_to} which is {education_in_not_in} your search range {education_search_range}"

education_narrative_01 = f"{education_narrative_search_criters}. The table below ranks qualifications in {ward_name}, {borough} and {city}. " + \
"The ranking is highest to lowest percentage from top to bottom. Where there is a difference in qualification the cell is shaded, a darker " + \
"shade denotes a difference between borough and ward. Values in [] give the percentage value."  

log.debug(f"\n{education_narrative_01}")

2022-08-24 13:40:27,154 [DEBUG] __main__: 
Using the latest qualifications data from 2011 which is outside your search range of 2012 to 2020. The table below ranks qualifications in Holloway, Islington and London. The ranking is highest to lowest percentage from top to bottom. Where there is a difference in qualification the cell is shaded, a darker shade denotes a difference between borough and ward. Values in [] give the percentage value.


In [23]:
# education_average_year_df
# education_by_borough_year_df
education_by_borough_ward_year_df


,YEAR,LAD,LAD_NAME,WARD_CODE,WARD_NAME,None,Level 1,Level 2,Level 3,Level 4,Other,Apprenticeship,FT Student 16 17,FT Student 18+,FT Student 18+ Employed,FT Student 18+ Unemployed
0,2011,E09000019,Islington,E05000375,Holloway,0.188078,0.084196,0.086884,0.111471,0.433394,0.086252,0.009724,0.023006,0.149182,0.03352,0.013361


#### Build the data and ranking table for ward, borough and city

In [24]:
## Borough & Ward
education_borough_ward_for_year = education_by_borough_ward_year_df.copy()
education_borough_ward_for_year = education_borough_ward_for_year.drop(["YEAR", "LAD", "LAD_NAME", "WARD_CODE", "WARD_NAME"], axis=1)
education_borough_ward_for_year = education_borough_ward_for_year.T.copy()
education_borough_ward_for_year.columns = ["PCT"]
education_borough_ward_for_year = education_borough_ward_for_year.sort_values(by=["PCT"], ascending=False)

## Borough
education_borough_for_year = education_by_borough_year_df[education_by_borough_year_df["LAD_NAME"] == borough].copy()
education_borough_for_year = education_borough_for_year.drop(["YEAR", "LAD", "LAD_NAME"], axis=1)
education_borough_for_year = education_borough_for_year.T.copy()
education_borough_for_year.columns = ["PCT"]
education_borough_for_year = education_borough_for_year.sort_values(by=["PCT"], ascending=False)

## City
education_city_for_year = education_average_year_df.copy()
education_city_for_year
education_city_for_year = education_city_for_year.drop(["YEAR"], axis=1)
education_city_for_year = education_city_for_year.T.copy()
education_city_for_year.columns = ["PCT"]
education_city_for_year = education_city_for_year.sort_values(by=["PCT"], ascending=False)

## Borough Ward
education_ward_pct_sorted = []
for index, row in education_borough_ward_for_year.iterrows():
    education     = index
    education_pct = row.values[0]*100
    education_name_pct_fmt = "{} - [{:,.2f}%]".format(education, education_pct)
    education_ward_pct_sorted.append(education_name_pct_fmt)

## Borough
education_borough_pct_sorted = []
for index, row in education_borough_for_year.iterrows():
    education     = index
    education_pct = row.values[0]*100
    education_name_pct_fmt = "{} - [{:,.2f}%]".format(education, education_pct)
    education_borough_pct_sorted.append(education_name_pct_fmt)

## City
education_city_pct_sorted = []
for index, row in     education_city_for_year.iterrows():
    education     = index
    education_pct = row.values[0]*100
    education_name_pct_fmt = "{} - [{:,.2f}%]".format(education, education_pct)
    education_city_pct_sorted.append(education_name_pct_fmt)

education_ward_borough_city_pct_ranked_merged = [education_ward_pct_sorted, education_borough_pct_sorted, education_city_pct_sorted]
education_ward_borough_city_pct_ranked_merged_df = pd.DataFrame(data=education_ward_borough_city_pct_ranked_merged)

## Rotate 
education_ward_borough_city_pct_ranked_merged_df = education_ward_borough_city_pct_ranked_merged_df.T
education_ward_borough_city_pct_ranked_merged_df.columns = [f"{ward_name}",f"{borough}",f"{city}"]
education_ward_borough_city_pct_ranked_merged_df.index   = [str(rank) for rank in range(1, len(education_ward_borough_city_pct_ranked_merged_df.index)+1)]

education_ward_borough_city_pct_ranked_merged_df

,Holloway,Islington,London
1,Level 4 - [43.34%],Level 4 - [48.07%],Level 4 - [39.40%]
2,None - [18.81%],None - [16.98%],None - [16.93%]
3,FT Student 18+ - [14.92%],FT Student 18+ - [10.71%],Level 2 - [11.56%]
4,Level 3 - [11.15%],Level 3 - [9.75%],Level 1 - [10.36%]
5,Level 2 - [8.69%],Level 2 - [8.40%],Level 3 - [10.32%]
6,Other - [8.63%],Level 1 - [8.04%],Other - [9.81%]
7,Level 1 - [8.42%],Other - [7.76%],FT Student 18+ - [7.85%]
8,FT Student 18+ Employed - [3.35%],FT Student 18+ Employed - [2.45%],FT Student 16 17 - [2.51%]
9,FT Student 16 17 - [2.30%],FT Student 16 17 - [1.86%],FT Student 18+ Employed - [2.42%]
10,FT Student 18+ Unemployed - [1.34%],Apprenticeship - [1.01%],Apprenticeship - [1.61%]


In [25]:
education_narrative_02 = ""

# education_level_4_borough_top_2


## Level 4 - top 2 then high
level_4_education_ward_top_2    = "Level 4" in str(education_ward_borough_city_pct_ranked_merged_df[0:2][ward_name])
level_4_education_borough_top_2 = "Level 4" in str(education_ward_borough_city_pct_ranked_merged_df[0:2][borough])
level_4_education_city_top_2 = "Level 4" in str(education_ward_borough_city_pct_ranked_merged_df[0:2][city])

## None    - top 4 then high
none_education_ward_top_4     = "None" in str(education_ward_borough_city_pct_ranked_merged_df[0:4][ward_name])
none_education_borough_top_4  = "None" in str(education_ward_borough_city_pct_ranked_merged_df[0:4][borough])
none_education_city_top_4  = "None" in str(education_ward_borough_city_pct_ranked_merged_df[0:4][city])

## FT Student 18+ top 3 then post school student area
student_education_ward_top_3     = "FT Student 18+" in str(education_ward_borough_city_pct_ranked_merged_df[0:3][ward_name])
student_education_borough_top_3  = "FT Student 18+" in str(education_ward_borough_city_pct_ranked_merged_df[0:3][borough])
student_education_city_top_3  = "FT Student 18+" in str(education_ward_borough_city_pct_ranked_merged_df[0:3][city])

## Build the narrative

## Level 4 education
level_4_high         = "" if level_4_education_ward_top_2 else " not"
level_4_high_similar = "similar" if level_4_education_ward_top_2 == level_4_education_borough_top_2 else "not the same"

# log.debug(f"none_education_ward_top_4:{none_education_ward_top_4}")
# log.debug(f"none_education_borough_top_4:{none_education_borough_top_4}")

## Non education
none_high_ward       = "high" if none_education_ward_top_4 else "low"
none_also            = "and also" if none_education_ward_top_4 == none_education_borough_top_4 else "but not"
none_high_borough    = "" if (none_education_ward_top_4 and none_education_borough_top_4) \
                       else " which ranks higher" if none_education_borough_top_4 else "which ranks lower"

## Student area narrative - ignored if not - Start
student_area_both_01 = "The ward" if (student_education_ward_top_3 and not student_education_borough_top_3) \
                  else " The borough" if (not student_education_ward_top_3 and student_education_borough_top_3) \
                  else " Both ward and borough"

student_area_both_02 = "the ward is a" if (student_education_ward_top_3 and not student_education_borough_top_3) \
                  else "the borough is a" if (not student_education_ward_top_3 and student_education_borough_top_3) \
                  else "both are"

student_area_both_03 = "" if (student_education_ward_top_3 and not student_education_borough_top_3) \
                  else "" if (not student_education_ward_top_3 and student_education_borough_top_3) \
                  else "s"

## Student area narrative - ignored if not - End

student_area         = "" if (not student_education_ward_top_3 and not student_education_borough_top_3) \
                          else f"{student_area_both_01} have high levels of full time 18+ students. Indicating that {student_area_both_02} student area{student_area_both_03}."

education_narrative_01 = f"For the borough {borough} and ward {ward_name}, the Level 4 education level (post secondary school including university) " + \
                         f"is{level_4_high} high, which is {level_4_high_similar} at the borough level." + \
                         f" Levels of no education (None) is {none_high_ward} for the ward {none_also} at the borough level{none_high_borough}." + \
                         f"{student_area}"

log.debug(f"\n{education_narrative_01}")


2022-08-24 13:40:27,701 [DEBUG] __main__: 
For the borough Islington and ward Holloway, the Level 4 education level (post secondary school including university) is high, which is similar at the borough level. Levels of no education (None) is high for the ward and also at the borough level. Both ward and borough have high levels of full time 18+ students. Indicating that both are student areas.


## Build the General Education table with formatting

In [26]:
colour_change = []
## 0 == no change in that cell
## 1 == shade 1 change
## 2 == shade 2 change

for index, row in education_ward_borough_city_pct_ranked_merged_df.iterrows():
    colour_change_row =[]
    
    ## Borough to City Check
    ward_val    = row.iloc[0].split(' - [')[0].strip()
    borough_val = row.iloc[1].split(' - [')[0].strip()
    city_val    = row.iloc[2].split(' - [')[0].strip()
    # log.debug(f"{ward_val}-{borough_val}-{city_val}")

    ward_val_cell_col = 0 if ward_val     == city_val else 1 if ward_val == borough_val else 2
    borough_val_col   = 0 if borough_val  == city_val else 1
    cityl_col         = 0
    
    colour_change_row.append(ward_val_cell_col)
    colour_change_row.append(borough_val_col)
    colour_change_row.append(cityl_col)
    colour_change.append(colour_change_row)

def format_ranking_row(row):
    ## Borough to City Check
    ward_val    = row.iloc[0].split(' - [')[0].strip()
    borough_val = row.iloc[1].split(' - [')[0].strip()
    city_val    = row.iloc[2].split(' - [')[0].strip()
    # log.debug(f"{ward_val}-{borough_val}-{city_val}")

    ward_val_cell_col = "" if ward_val     == city_val else "background-color: #EAFAF1" if ward_val == borough_val else "background-color: #D5F5E3"
    borough_val_col   = "" if borough_val  == city_val else "background-color: #EAFAF1"
    # log.debug(r[1])
    return [ward_val_cell_col] + [borough_val_col] + [""]


from IPython.display import HTML
styles = [
  dict(selector="tr", props=[("font-size", "110%"),
                             ("text-align", "right")])
]

education_ward_borough_city_pct_ranked_merged_df_html = (education_ward_borough_city_pct_ranked_merged_df.style.set_table_styles(styles).apply(format_ranking_row, axis=1))


In [27]:
education_ward_borough_city_pct_ranked_merged_df_html

,Holloway,Islington,London
1,Level 4 - [43.34%],Level 4 - [48.07%],Level 4 - [39.40%]
2,None - [18.81%],None - [16.98%],None - [16.93%]
3,FT Student 18+ - [14.92%],FT Student 18+ - [10.71%],Level 2 - [11.56%]
4,Level 3 - [11.15%],Level 3 - [9.75%],Level 1 - [10.36%]
5,Level 2 - [8.69%],Level 2 - [8.40%],Level 3 - [10.32%]
6,Other - [8.63%],Level 1 - [8.04%],Other - [9.81%]
7,Level 1 - [8.42%],Other - [7.76%],FT Student 18+ - [7.85%]
8,FT Student 18+ Employed - [3.35%],FT Student 18+ Employed - [2.45%],FT Student 16 17 - [2.51%]
9,FT Student 16 17 - [2.30%],FT Student 16 17 - [1.86%],FT Student 18+ Employed - [2.42%]
10,FT Student 18+ Unemployed - [1.34%],Apprenticeship - [1.01%],Apprenticeship - [1.61%]


In [28]:
education_borough_ward_for_year_name_sorted = education_borough_ward_for_year.sort_index(ascending=True)
education_borough_for_year_name_sorted      = education_borough_for_year.sort_index(ascending=True)
education_city_for_year_name_sorted         = education_city_for_year.sort_index(ascending=True)

education_borough_ward_for_year_name_sorted = education_borough_ward_for_year
education_borough_for_year_name_sorted      = education_borough_for_year
education_city_for_year_name_sorted         = education_city_for_year


## Borough Ward
education_ward_name_sorted = []
for index, row in education_borough_ward_for_year_name_sorted.iterrows():
    education_pct = row.values[0]*100
    education_pct_fmt = "{:,.2f}%".format(education_pct)
    education_ward_name_sorted.append(education_pct_fmt)

## Borough
education_borough_name_sorted = []
for index, row in education_borough_for_year_name_sorted.iterrows():
    education_pct = row.values[0]*100
    education_pct_fmt = "{:,.2f}%".format(education_pct)
    education_borough_name_sorted.append(education_pct_fmt)

## City
education_city_name_sorted = []
for index, row in education_city_for_year_name_sorted.iterrows():
    education_pct = row.values[0]*100
    education_pct_fmt = "{:,.2f}%".format(education_pct)
    education_city_name_sorted.append(education_pct_fmt)

education_ward_borough_city_pct_name_merged = [education_ward_name_sorted, education_borough_name_sorted, education_city_name_sorted]
education_ward_borough_city_pct_name_merged_df = pd.DataFrame(data=education_ward_borough_city_pct_name_merged)

## Rotate 
education_ward_borough_city_pct_name_merged_df = education_ward_borough_city_pct_name_merged_df.T
education_ward_borough_city_pct_name_merged_df.columns = [f"{ward_name}",f"{borough}",f"{city}"]
education_ward_borough_city_pct_name_merged_df.index   = education_borough_ward_for_year_name_sorted.index

education_ward_borough_city_pct_name_merged_df

,Holloway,Islington,London
Level 4,43.34%,48.07%,39.40%
None,18.81%,16.98%,16.93%
FT Student 18+,14.92%,10.71%,11.56%
Level 3,11.15%,9.75%,10.36%
Level 2,8.69%,8.40%,10.32%
Other,8.63%,8.04%,9.81%
Level 1,8.42%,7.76%,7.85%
FT Student 18+ Employed,3.35%,2.45%,2.51%
FT Student 16 17,2.30%,1.86%,2.42%
FT Student 18+ Unemployed,1.34%,1.01%,1.61%


In [29]:
def format_pct_row(row):
    
    # log.debug(f"index:{row.name}")
    
    ## Borough to City 
    # name        = row.name
    ward_val    = float(row.iloc[0].split("%")[0].strip())
    borough_val = float(row.iloc[1].split("%")[0].strip())
    city_val    = float(row.iloc[2].split("%")[0].strip())
    
    ## It's more than the city so should be green
    if ward_val >= city_val:
        # log.debug("Ward more so Green")
        diff = ward_val - city_val
        ward_val_cell_col = ""                          if (diff) < 1.0 else \
                            "background-color: #EAFAF1" if (diff) < 2.0 else \
                            "background-color: #D5F5E3" if (diff) < 3.0 else \
                            "background-color: #ABEBC6" if (diff) < 4.0 else \
                            "background-color: #82E0AA" if (diff) < 5.0 else \
                            "background-color: #58D68D"

    ## It's less than, so should be red
    else:
        # log.debug("Ward less so Red")
        #-ve then red shades
        diff = city_val - ward_val
        ward_val_cell_col = ""                          if (diff) < 1.0 else \
                            "background-color: #F5EEF8" if (diff) < 2.0 else \
                            "background-color: #EBDEF0" if (diff) < 3.0 else \
                            "background-color: #D7BDE2" if (diff) < 4.0 else \
                            "background-color: #C39BD3" if (diff) < 5.0 else \
                            "background-color: #AF7AC5"

    ## It's more than the city so should be green
    if borough_val >= city_val:
        # log.debug("Borough more so Green")
        #+ve then green shades
        diff = borough_val - city_val
        borough_val_col   = ""                          if (diff) < 1.0 else \
                            "background-color: #EAFAF1" if (diff) < 2.0 else \
                            "background-color: #D5F5E3" if (diff) < 3.0 else \
                            "background-color: #ABEBC6" if (diff) < 4.0 else \
                            "background-color: #82E0AA" if (diff) < 5.0 else \
                            "background-color: #58D68D"


    ## It's less than, so should be red
    else:
        # log.debug("Borough less so Red")
        #-ve then red shades
        diff = city_val - borough_val
        borough_val_col   = ""                          if (diff) < 1.0 else \
                            "background-color: #F5EEF8" if (diff) < 2.0 else \
                            "background-color: #EBDEF0" if (diff) < 3.0 else \
                            "background-color: #D7BDE2" if (diff) < 4.0 else \
                            "background-color: #C39BD3" if (diff) < 5.0 else \
                            "background-color: #AF7AC5"

    return [ward_val_cell_col] + [borough_val_col] + [""]


from IPython.display import HTML
styles = [
  dict(selector="tr", props=[("font-size", "110%"),
                             ("text-align", "right")])
]

education_ward_borough_city_pct_name_merged_df_html = (education_ward_borough_city_pct_name_merged_df.style.set_table_styles(styles).apply(format_pct_row, axis=1))
# education_ward_borough_city_pct_name_merged_df_html

In [30]:
education_narrative_02 = f"The table below shows the percentage levels of qualifications achieved in {ward_name}, {borough} and {city} for the year {education_year_to}" + \
". Value shading indicates a difference from the city level from 1 to 5 percent in 1 percent intervals." + \
" The shade darkens with an increase in difference. Increases and decreases use different colours for clarity only."

log.debug(f"\n{education_narrative_02}")

2022-08-24 13:40:28,423 [DEBUG] __main__: 
The table below shows the percentage levels of qualifications achieved in Holloway, Islington and London for the year 2011. Value shading indicates a difference from the city level from 1 to 5 percent in 1 percent intervals. The shade darkens with an increase in difference. Increases and decreases use different colours for clarity only.


In [31]:
education_ward_borough_city_pct_name_merged_df_html

,Holloway,Islington,London
Level 4,43.34%,48.07%,39.40%
None,18.81%,16.98%,16.93%
FT Student 18+,14.92%,10.71%,11.56%
Level 3,11.15%,9.75%,10.36%
Level 2,8.69%,8.40%,10.32%
Other,8.63%,8.04%,9.81%
Level 1,8.42%,7.76%,7.85%
FT Student 18+ Employed,3.35%,2.45%,2.51%
FT Student 16 17,2.30%,1.86%,2.42%
FT Student 18+ Unemployed,1.34%,1.01%,1.61%


# REPORT PAGE

In [32]:
print(education_narrative_01)

For the borough Islington and ward Holloway, the Level 4 education level (post secondary school including university) is high, which is similar at the borough level. Levels of no education (None) is high for the ward and also at the borough level. Both ward and borough have high levels of full time 18+ students. Indicating that both are student areas.


In [33]:
education_ward_borough_city_pct_ranked_merged_df_html

,Holloway,Islington,London
1,Level 4 - [43.34%],Level 4 - [48.07%],Level 4 - [39.40%]
2,None - [18.81%],None - [16.98%],None - [16.93%]
3,FT Student 18+ - [14.92%],FT Student 18+ - [10.71%],Level 2 - [11.56%]
4,Level 3 - [11.15%],Level 3 - [9.75%],Level 1 - [10.36%]
5,Level 2 - [8.69%],Level 2 - [8.40%],Level 3 - [10.32%]
6,Other - [8.63%],Level 1 - [8.04%],Other - [9.81%]
7,Level 1 - [8.42%],Other - [7.76%],FT Student 18+ - [7.85%]
8,FT Student 18+ Employed - [3.35%],FT Student 18+ Employed - [2.45%],FT Student 16 17 - [2.51%]
9,FT Student 16 17 - [2.30%],FT Student 16 17 - [1.86%],FT Student 18+ Employed - [2.42%]
10,FT Student 18+ Unemployed - [1.34%],Apprenticeship - [1.01%],Apprenticeship - [1.61%]


In [34]:
print(education_narrative_02)

The table below shows the percentage levels of qualifications achieved in Holloway, Islington and London for the year 2011. Value shading indicates a difference from the city level from 1 to 5 percent in 1 percent intervals. The shade darkens with an increase in difference. Increases and decreases use different colours for clarity only.


In [35]:
education_ward_borough_city_pct_name_merged_df_html

,Holloway,Islington,London
Level 4,43.34%,48.07%,39.40%
None,18.81%,16.98%,16.93%
FT Student 18+,14.92%,10.71%,11.56%
Level 3,11.15%,9.75%,10.36%
Level 2,8.69%,8.40%,10.32%
Other,8.63%,8.04%,9.81%
Level 1,8.42%,7.76%,7.85%
FT Student 18+ Employed,3.35%,2.45%,2.51%
FT Student 16 17,2.30%,1.86%,2.42%
FT Student 18+ Unemployed,1.34%,1.01%,1.61%


# REPORT SECTION TEST

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-08-24 13:40:29,509 [INFO] lib.db_tools: DATABASE CONNECTIVITY
2022-08-24 13:40:29,510 [DEBUG] root: Retrieving qualifications min max year
2022-08-24 13:40:29,547 [DEBUG] managers.sections.sd_report_section_05_Education:    MAX_YEAR  MIN_YEAR
0      2011      2011
2022-08-24 13:40:29,549 [DEBUG] managers.sections.sd_report_section_05_Education: education_year_min:2011
2022-08-24 13:40:29,550 [DEBUG] managers.sections.sd_report_section_05_Education: education_year_max:2011
2022-08-24 13:40:29,551 [DEBUG] managers.sections.sd_report_section_05_Education: education orig_year_to    :2020
2022-08-24 13:40:29,552 [DEBUG] managers.sections.sd_report_section_05_Education: education search_year_to  :2011
2022-08-24 13:40:29,553 [DEBUG] root: education_ratio_average_years


C:\DISTRIBUTIONS\Python\Python3_10_2\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\DISTRIBUTIONS\Python\Python3_10_2\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


2022-08-24 13:40:30,692 [DEBUG] root: education_ratio_by_borough_years


C:\DISTRIBUTIONS\Python\Python3_10_2\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


2022-08-24 13:40:31,669 [DEBUG] root: education_ratio_by_borough_ward_years


C:\DISTRIBUTIONS\Python\Python3_10_2\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\DISTRIBUTIONS\Python\Python3_10_2\lib\site-packages\dataframe_image\_pandas_accessor.py:69: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html = '<div>' + obj.render() + '</div>'
C:\DISTRIBUTIONS\Python\Python3_10_2\lib\site-packages\dataframe_image\_pandas_accessor.py:69: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html = '<div>' + obj.render() + '</div>'


2022-08-24 13:40:34,147 [DEBUG] __main__: {'education_narrative_01': 'For the borough Islington and ward Holloway, the Level 4 education level (post secondary school including university) is high, which is similar at the borough level. Levels of no education (None) is high for the ward and also at the borough level. Both ward and borough have high levels of full time 18+ students. Indicating that both are student areas.', 'education_ranking_display_table': './reports/generation/images/XX_education_ranking_display_table_London_Islington_Holloway.png', 'educataion_comparison_display_table': './reports/generation/images/XX_education_comparison_display_table_London_Islington_Holloway.png', 'education_narrative_02': 'The table below shows the percentage levels of qualifications achieved in Holloway, Islington and London for the year 2011. Value shading indicates a difference from the city level from 1 to 5 percent in 1 percent intervals. The shade darkens with an increase in difference. Inc

In [84]:
def format_pct_row(row, x):
    
    # log.debug(f"index:{row.name}")
    
    ## Borough to City 
    # name        = row.name
    ward_val    = float(row.iloc[1].split("%")[0].strip())
    borough_val = float(row.iloc[2].split("%")[0].strip())
    city_val    = float(row.iloc[3].split("%")[0].strip())
    
    ## It's more than the city so should be green
    if ward_val >= city_val:
        # log.debug("Ward more so Green")
        diff = ward_val - city_val
        ward_val_cell_col = ""                          if (diff) < 1.0 else \
                            "EAFAF1" if (diff) < 2.0 else \
                            "D5F5E3" if (diff) < 3.0 else \
                            "ABEBC6" if (diff) < 4.0 else \
                            "82E0AA" if (diff) < 5.0 else \
                            "58D68D"

    ## It's less than, so should be red
    else:
        # log.debug("Ward less so Red")
        #-ve then red shades
        diff = city_val - ward_val
        ward_val_cell_col = ""                          if (diff) < 1.0 else \
                            "F5EEF8" if (diff) < 2.0 else \
                            "EBDEF0" if (diff) < 3.0 else \
                            "D7BDE2" if (diff) < 4.0 else \
                            "C39BD3" if (diff) < 5.0 else \
                            "AF7AC5"

    ## It's more than the city so should be green
    if borough_val >= city_val:
        # log.debug("Borough more so Green")
        #+ve then green shades
        diff = borough_val - city_val
        borough_val_col   = ""                          if (diff) < 1.0 else \
                            "EAFAF1" if (diff) < 2.0 else \
                            "D5F5E3" if (diff) < 3.0 else \
                            "ABEBC6" if (diff) < 4.0 else \
                            "82E0AA" if (diff) < 5.0 else \
                            "58D68D"


    ## It's less than, so should be red
    else:
        # log.debug("Borough less so Red")
        #-ve then red shades
        diff = city_val - borough_val
        borough_val_col   = ""                          if (diff) < 1.0 else \
                            "F5EEF8" if (diff) < 2.0 else \
                            "EBDEF0" if (diff) < 3.0 else \
                            "D7BDE2" if (diff) < 4.0 else \
                            "C39BD3" if (diff) < 5.0 else \
                            "AF7AC5"

    x.append(["", ward_val_cell_col, borough_val_col,""])


from IPython.display import HTML
styles = [
  dict(selector="tr", props=[("font-size", "110%"),
                             ("text-align", "right")])
]

docx_education_ward_borough_city_pct_name_merged_df = education_ward_borough_city_pct_name_merged_df.copy()
docx_education_ward_borough_city_pct_name_merged_df["Level"] = docx_education_ward_borough_city_pct_name_merged_df.index
docx_education_ward_borough_city_pct_name_merged_df = docx_education_ward_borough_city_pct_name_merged_df[["Level", "Holloway", "Islington", "London"]]

cell_shading = [["","",""]]
docx_education_ward_borough_city_pct_name_merged_df.apply(lambda row: format_pct_row(row, cell_shading), axis=1)
# education_ward_borough_city_pct_name_merged_df_html
log.debug(cell_shading)

# for row in cell_shading:
#   for cell in row:
    
    # log.debug(cell)

2022-08-24 20:08:48,783 [DEBUG] __main__: [['', '', ''], ['', 'ABEBC6', '58D68D', ''], ['', 'EAFAF1', '', ''], ['', 'ABEBC6', '', ''], ['', '', '', ''], ['', 'F5EEF8', 'F5EEF8', ''], ['', 'F5EEF8', 'F5EEF8', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', '']]


# WORD DOC

In [24]:
import docx
import pandas as pd
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml

def delete_row():
  """
  
  """
  if "table_cell" not in globals():
    log.error("Can not reference table_cell")
  else:
    table_cell = globals()["table_cell"]
    parent = table_cell._parent
    
    row_to_delete = None
    for row in parent.rows:
      for cell in row.cells:
        if cell._element == table_cell._element:
          row_to_delete = row
    
    if row_to_delete != None:
      parent._tbl.remove(row_to_delete._tr)
    
    
from copy import deepcopy

def copy_table_after(table, paragraph):
  tbl, p = table._tbl, paragraph._p
  new_tbl = deepcopy(tbl)
  p.addnext(new_tbl)

def include_table(data, shading=None, style=None, columns=None):
  """

  """
  if "table_cell" not in globals():
    log.error("Can not reference table_cell")
  else:
    table_cell = globals()["table_cell"]

    document_styles = table_cell._parent._parent._parent.styles

    ## Make a note of what's there, we need to remove it later
    existing_paragraphs = table_cell.paragraphs
    
    ## If we want shading then make a dataframe with the 
    ## original data column
    if shading != None:
        log.debug(data.columns.tolist())
        shading_df = pd.DataFrame(data=shading, columns=[data.columns.tolist()])

    ## make a frame from the columns - should be able to re-order here too
    if columns != None:
        data = data[columns]
        
    ## Create the new table to populate with our data
    dataframe_table = table_cell.add_table(data.shape[0]+1, data.shape[1])

    ## Remove the existing paragraph so it fits properly
    for paragraph in existing_paragraphs:
        p = paragraph._element
        p.getparent().remove(p)
        p._p = p._element = None
        
    # p = table_cell.paragraphs[0]._element
    # p.getparent().remove(p)
    # p._p = p._element = None

    ## Needed?
    dataframe_table.allow_autofit = True
    dataframe_table.autofit = True

    
    ## Add any table style - the table style MUST exist in the word doc being used
    if style != None:
        for style in document_styles:
            log.debug(style.name)
            
        dataframe_table.style = style

    ## Populate the table with our dataframe
    
    ### add the header rows.
    for j in range(data.shape[-1]):
        dataframe_table.cell(0,j).text = data.columns[j]

    ### add the rest of the data frame
    for i in range(data.shape[0]):
        for j in range(data.shape[-1]):
            dataframe_table.cell(i+1,j).text = str(data.values[i,j])    

    ## Add any cell colouring
    if shading != None:
        
        ## If we want to change the columns then
        if columns != None:
            data = data[columns]
            shading_df = shading_df[columns]
        
        ## We might have re-ordered the columns so remake the list
        shading = shading_df.values.tolist()
        
        s_r_idx = 0                          # Style row index
        for shading_row in shading:          # Style rows LOOP
            s_c_idx = 0                      # Style cell index
            for shading_cell in shading_row: # Style cells LOOP
                if shading_cell:             # There's a shading in there, let's use it!
                    ## Remove # in colour
                    shading_cell = shading_cell.replace("#", "")
                    ## Create the shading element and apply it
                    shading_element = parse_xml(r'<w:shd {} w:fill="{}"/>'.format(nsdecls("w"), shading_cell))
                    dataframe_table.rows[s_r_idx+1].cells[s_c_idx]._tc.get_or_add_tcPr().append(shading_element)
                s_c_idx += 1
            
            s_r_idx += 1

# # open an existing document
table_document = docx.Document("./reports/templates/test.docx")

## Loop through the table cells - only one in this test doc - mimicing the existing code
tables = table_document.tables
for table in tables:
    for row in table.rows:
        for cell in row.cells:
            globals()["table_cell"] = cell
            

data = [["Neal","07768221573"],["Sally", "07956697060"]]            
dataframe = pd.DataFrame(data=data, columns=["Name", "Number"])

shading = [["#EAFAF1","#F5EEF8"],["#EAFAF1","#F5EEF8"]]
# shading_df = pd.DataFrame(data=shading, columns = dataframe.columns)

if 1 == 1:
    style = "List Table 3 Accent 5"
    # style = None

    columns = ["Name", "Number"]
    # columns = None


    include_table(dataframe, shading=shading, style=style, columns=columns)            

    table_document.save("./reports/generation/documents/test_with_tableX.docx")       
    
# log.debug(shading_df.values.tolist())

2022-08-25 13:24:17,755 [DEBUG] managers.sd_report_manager_new: ['Name', 'Number']
2022-08-25 13:24:17,761 [DEBUG] managers.sd_report_manager_new: Normal
2022-08-25 13:24:17,762 [DEBUG] managers.sd_report_manager_new: Default Paragraph Font
2022-08-25 13:24:17,763 [DEBUG] managers.sd_report_manager_new: Normal Table
2022-08-25 13:24:17,764 [DEBUG] managers.sd_report_manager_new: No List
2022-08-25 13:24:17,765 [DEBUG] managers.sd_report_manager_new: Table Grid
2022-08-25 13:24:17,766 [DEBUG] managers.sd_report_manager_new: Grid Table 1 Light
2022-08-25 13:24:17,767 [DEBUG] managers.sd_report_manager_new: Grid Table 4
2022-08-25 13:24:17,768 [DEBUG] managers.sd_report_manager_new: List Table 3 Accent 5


In [ ]:
dataframe

In [40]:
%load_ext autoreload

%autoreload 3

from managers import sd_report_manager_new as report_man

report_context_new = {"name_and_numbers":dataframe,
                      "name_and_numbers_shading":shading,
                      "template_processor_file_name":"./reports/processors/sd_test_include_table.json",
                      "report_option":1} 

log.debug("Started")

generated_report = report_man.generate_report("YY"
                                            , report_context = report_context_new
                                            , properties = properties)

log.debug("Finished")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-08-25 13:41:11,246 [DEBUG] managers.sd_report_manager_new: Started
2022-08-25 13:41:11,261 [DEBUG] managers.sd_report_manager_new: Normal
2022-08-25 13:41:11,262 [DEBUG] managers.sd_report_manager_new: Default Paragraph Font
2022-08-25 13:41:11,262 [DEBUG] managers.sd_report_manager_new: Normal Table
2022-08-25 13:41:11,264 [DEBUG] managers.sd_report_manager_new: No List
2022-08-25 13:41:11,265 [DEBUG] managers.sd_report_manager_new: Table Grid
2022-08-25 13:41:11,266 [DEBUG] managers.sd_report_manager_new: Grid Table 1 Light
2022-08-25 13:41:11,268 [DEBUG] managers.sd_report_manager_new: Grid Table 4
2022-08-25 13:41:11,270 [DEBUG] managers.sd_report_manager_new: List Table 3 Accent 5
2022-08-25 13:41:11,271 [DEBUG] managers.sd_report_manager_new: List Table 4 Accent 5
2022-08-25 13:41:11,272 [DEBUG] managers.sd_report_manager_new: List Table 4 Accent 1
2022-08-25 13:41:11,274 [DEBUG] managers

In [85]:
import docx
import pandas as pd
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml

# # i am not sure how you are getting your data, but you said it is a
# # pandas data frame
df = docx_education_ward_borough_city_pct_name_merged_df

# # open an existing document
doc = docx.Document("./reports/templates/test.docx")


for style in doc.styles:
    log.debug(style.name)

# add a table to the end and create a reference variable
# extra row is so we can add the header row
t = doc.add_table(df.shape[0]+1, df.shape[1],  style="List Table 3 Accent 5")

t.allow_autofit = True
t.autofit = True
# t.style = "Light Grid"

# add the header rows.
for j in range(df.shape[-1]):
    t.cell(0,j).text = df.columns[j]

# add the rest of the data frame
for i in range(df.shape[0]):
    for j in range(df.shape[-1]):
        t.cell(i+1,j).text = str(df.values[i,j])


r_idx = 0
for row in cell_shading:
  c_idx = 0
  for cell in row:
    if cell :
        shading_elm_1 = parse_xml(r'<w:shd {} w:fill="{}"/>'.format(nsdecls("w"), cell))
        t.rows[r_idx].cells[c_idx]._tc.get_or_add_tcPr().append(shading_elm_1)
    
    c_idx += 1
  
  r_idx += 1

# save the doc
doc.save("./reports/templates/test_with_table.docx")

2022-08-24 20:08:54,184 [DEBUG] __main__: Normal
2022-08-24 20:08:54,185 [DEBUG] __main__: Default Paragraph Font
2022-08-24 20:08:54,186 [DEBUG] __main__: Normal Table
2022-08-24 20:08:54,187 [DEBUG] __main__: No List
2022-08-24 20:08:54,188 [DEBUG] __main__: Table Grid
2022-08-24 20:08:54,189 [DEBUG] __main__: Grid Table 1 Light
2022-08-24 20:08:54,189 [DEBUG] __main__: Grid Table 4
2022-08-24 20:08:54,190 [DEBUG] __main__: List Table 3 Accent 5


In [41]:
docx_education_ward_borough_city_pct_name_merged_df

,Holloway,Islington,London
Level 4,43.34%,48.07%,39.40%
None,18.81%,16.98%,16.93%
FT Student 18+,14.92%,10.71%,11.56%
Level 3,11.15%,9.75%,10.36%
Level 2,8.69%,8.40%,10.32%
Other,8.63%,8.04%,9.81%
Level 1,8.42%,7.76%,7.85%
FT Student 18+ Employed,3.35%,2.45%,2.51%
FT Student 16 17,2.30%,1.86%,2.42%
FT Student 18+ Unemployed,1.34%,1.01%,1.61%
